In [1]:
# Classes

class Card():
    '''A class to manage a single card'''
    
    def __init__(self,
                 value:str = None,
                 suit:str = None):
        
        self.value = value
        self.suit = suit
        self.name = self.value + ' of ' + self.suit
    
class Pile():
    '''A class to manage an ordered set of cards'''
    
    def __init__(self,
                 name:str = None,
                 card_list:list = None):
        
        self.name = name
        
        if card_list is None:
            self.card_list = []
        else:
            self.card_list = card_list
        
    def shuffle(self):
        # This method is called to shuffle the pile
        
        random.shuffle(self.card_list)
        
class Hand(Pile):
    '''A class to manage a hand of cards, child class to Pile'''
    
    def __init__(self,
                 name:str = None,
                 card_list:list = None,
                 owner:str = None):
        
        Pile.__init__(self,name,card_list)
        self.name = owner + "'s Hand"
        self.owner = owner
        
    def draw(self,
             pile:str = None,
             number:int = 1):
        # This method is called to draw a number of cards to the hand from the draw pile
        
        self.card_list.extend(pile.card_list[-number:])
        del pile.card_list[-number:]
        
    def play_book(self,
                 pile:str = None,
                 played_cards:list = None):
        # This method is called to play a book of cards from the hand to the discard pile
        
        for card in played_cards:
            self.card_list.remove(card)
            pile.card_list.append(card)
            
class Scoreboard():
    '''A class to manage the overall game state'''
    
    def __init__(self,
                players:list = None,
                played_books:list = None):
        
        if players is None:
            self.players = []
        else:
            self.players = players
        
        self.score = {}
        for player in self.players:
            self.score[player] = 0
        
        if played_books is None:
            self.played_books = []
        else:
            self.books = played_books
            
    def update_score(self,
                    player:str = None,
                    book:str = None):
        # This method is called when a book has been played and the scoreboard needs to be updated
        
        self.score[player] += 1
        self.played_books.append(book)
    
    def print_score(self):
        # This method is called to print a summary of the game state
        
        print('Score  Player')
        for player in sorted(self.score, key = self.score.get, reverse = True):
            print(str(self.score[player]) + '      ' + player)
        
        

In [2]:
# Game initialization  
import random

def startgame(player_list = ['player1','player2']):
    # Set variables and check for valid player count
    cards, hands = [], []
    player_count = len(player_list)
        
    if player_count > 1 and player_count < 4:
        starting_hand_size = 7
    elif player_count > 3 and player_count < 7:
        starting_hand_size = 5
    else:
        return print('There must be between two and six players.')
    
    suits = {0:"Hearts", 1:"Diamonds", 2:"Clubs", 3:"Spades"}
    values = {0:"Two", 1:"Three", 2:"Four", 3:"Five", 4:"Six", 5:"Seven", 6:"Eight",
              7:"Nine", 8:"Ten", 9:"Jack", 10:"Queen", 11:"King", 12:"Ace"}
    
    # Create the 52 playing cards
    for s in range(4):
        for v in range(13):
            cards.append(Card(value = values[v], suit = suits[s]))
    
    # Create piles, scoreboards, and hands
    draw_pile = Pile(name = "Draw Pile", card_list = cards.copy())
    discard_pile = Pile(name = "Discard Pile")
    scoreboard = Scoreboard(players = player_list)
    for player in scoreboard.players:
        hands.append(Hand(owner = player))
    
    # Shuffle up and deal
    draw_pile.shuffle()
    for hand in hands:
        hand.draw(pile = draw_pile, number = starting_hand_size)
    
    return cards, draw_pile, discard_pile, scoreboard, hands

In [3]:
#Test area
cards, draw_pile, discard_pile, scoreboard, hands = startgame(['Alex','Louis','Erin'])

print('The Draw Pile has ' + str(len(draw_pile.card_list)) + ' cards and the top card is:')
print('  ' + draw_pile.card_list[-1].value + ' of ' + draw_pile.card_list[-1].suit)
print('\n')
for player in hands:
    print(player.name)
    for card in player.card_list:
        print('  ' + card.name)
    print('\n')

hands[0].play_book(pile = discard_pile, played_cards = hands[0].card_list[-4:])
print('After playing a "book", Alex has ' + str(len(hands[0].card_list)) + ' cards.')
print('Discard Pile:')
for card in discard_pile.card_list:
    print('  ' + card.name)
print('\n')
scoreboard.print_score()


The Draw Pile has 31 cards and the top card is:
  Jack of Hearts


Alex's Hand
  Nine of Diamonds
  Two of Spades
  Queen of Spades
  King of Clubs
  Seven of Diamonds
  Three of Clubs
  Queen of Hearts


Louis's Hand
  Seven of Clubs
  Eight of Clubs
  Ten of Hearts
  Ten of Diamonds
  Three of Hearts
  Jack of Diamonds
  King of Diamonds


Erin's Hand
  Five of Spades
  Jack of Spades
  Seven of Spades
  Four of Hearts
  Eight of Diamonds
  Five of Clubs
  Eight of Spades


After playing a "book", Alex has 3 cards.
Discard Pile:
  King of Clubs
  Seven of Diamonds
  Three of Clubs
  Queen of Hearts


Score  Player
0      Alex
0      Louis
0      Erin
